In [1]:
# LSST stack imports
from lsst.daf.butler import Butler 
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import numpy as np

repo = "/repo/main"
collection = "LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478"
instrument = "LSSTComCam"

afwDisplay.setDefaultBackend('firefly')
display1 = afwDisplay.Display(frame=1)
display1.getClient().get_firefly_url()

butler = Butler(repo, collections=collection)

In [15]:
source_id = 191825315813654536
visit_id = 2024120500111
detector_id = 0

In [16]:
diaSource = butler.get(
    'goodSeeingDiff_diaSrc',
    dataId={'visit': visit_id, 'instrument': 'LSSTComCam', 'detector': detector_id})

diaSourceTable = butler.get(
    'goodSeeingDiff_diaSrcTable',
    dataId={'visit': visit_id, 'instrument': 'LSSTComCam', 'detector': detector_id})

x_position = diaSource[diaSource['id'] ==source_id]['base_SdssCentroid_x']
y_position = diaSource[diaSource['id'] ==source_id]['base_SdssCentroid_y']
print(x_position, y_position)

[3637.] [181.]


In [17]:
display1.erase()
butler = Butler(repo, collections=collection)
image = butler.get('goodSeeingDiff_differenceExp', visit=visit_id, detector=detector_id, instrument="LSSTComCam")
display1.mtv(image)

with display1.Buffering():
    for key, record in diaSourceTable.iterrows():
        display1.dot('+', record['x'], record['y'], size=20, ctype='red')

with display1.Buffering():
    for record in diaSource:
        footprint = record.getFootprint()
        for peak in footprint.getPeaks()[::10]:
            display1.dot('o', peak['f_x'], peak['f_y'], size=10, ctype='orange')

display1.dot('x', x_position, y_position, size=200, ctype='magenta')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/ipykernel/__main__.py", line 5, in <module>
    app.launch_new_instance()
  File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-pa

AttributeError: _ARRAY_API not found

In [18]:
schema = diaSource.schema
schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['F'](name="coord_raErr", doc="1-sigma uncertainty on ra", units="rad"), Key<F>(offset=32, nElements=1)),
    (Field['F'](name="coord_decErr", doc="1-sigma uncertainty on dec", units="rad"), Key<F>(offset=36, nElements=1)),
    (Field['F'](name="coord_ra_dec_Cov", doc="uncertainty covariance between ra and dec", units="rad rad"), Key<F>(offset=40, nElements=1)),
    (Field['Flag'](name="is_negative", doc="set if source was detected as significantly negative"), Key['Flag'](offset=48, bit=0)),
    (Field['I'](name="deblend_nChild", doc="Number of children this object has (defaults to 0)"), Key<I>(of

In [6]:
x_position

array([3709.])

In [7]:
y_position

array([1501.])

In [11]:
flag_columns = [item.field.getName() for item in schema if "flag" in item.field.getName()]
flag_columns

['flags_negative',
 'base_SdssCentroid_flag',
 'base_SdssCentroid_flag_edge',
 'base_SdssCentroid_flag_noSecondDerivative',
 'base_SdssCentroid_flag_almostNoSecondDerivative',
 'base_SdssCentroid_flag_notAtMaximum',
 'base_SdssCentroid_flag_near_edge',
 'base_SdssCentroid_flag_resetToPeak',
 'base_SdssCentroid_flag_badError',
 'ip_diffim_DipoleFit_flag_classification',
 'ip_diffim_DipoleFit_flag_classificationAttempted',
 'ip_diffim_DipoleFit_flag',
 'ip_diffim_DipoleFit_flag_edge',
 'ext_shapeHSM_HsmPsfMoments_flag',
 'ext_shapeHSM_HsmPsfMoments_flag_no_pixels',
 'ext_shapeHSM_HsmPsfMoments_flag_not_contained',
 'ext_shapeHSM_HsmPsfMoments_flag_parent_source',
 'ext_shapeHSM_HsmPsfMoments_flag_galsim',
 'ext_shapeHSM_HsmPsfMoments_flag_invalid_param',
 'ext_shapeHSM_HsmPsfMoments_flag_edge',
 'ext_shapeHSM_HsmPsfMoments_flag_no_psf',
 'ext_shapeHSM_HsmSourceMoments_flag',
 'ext_shapeHSM_HsmSourceMoments_flag_no_pixels',
 'ext_shapeHSM_HsmSourceMoments_flag_not_contained',
 'ext_shapeH

In [12]:
import pandas as pd

# Convert SourceCatalog to an Astropy Table and then to a pandas DataFrame
diaSource_df = diaSource.asAstropy().to_pandas()

# Perform filtering and column selection
result_df = diaSource_df[flag_columns]


# Filter rows where all columns are False
filtered_df = result_df[(result_df == False).all(axis=1)]

diaSource_df.loc[filtered_df.index.values][['base_SdssCentroid_x','base_SdssCentroid_y']]

,base_SdssCentroid_x,base_SdssCentroid_y
53,1101.997875,110.691367
78,958.679177,189.494695
203,1657.609724,421.581655
311,665.328546,583.450722
464,956.009443,856.041565
478,691.097750,893.994181
669,110.927827,1329.726241
691,113.823884,1361.266867
705,22.535097,1403.204297
751,2352.996124,1519.034524


In [13]:
print(list(diaSource_df.columns))

['id', 'coord_ra', 'coord_dec', 'parent', 'coord_raErr', 'coord_decErr', 'coord_ra_dec_Cov', 'flags_negative', 'deblend_nChild', 'deblend_deblendedAsPsf', 'deblend_psfCenter_x', 'deblend_psfCenter_y', 'deblend_psf_instFlux', 'deblend_tooManyPeaks', 'deblend_parentTooBig', 'deblend_masked', 'deblend_failed', 'deblend_skipped', 'deblend_rampedTemplate', 'deblend_patchedTemplate', 'deblend_hasStrayFlux', 'deblend_peak_center_x', 'deblend_peak_center_y', 'deblend_peakId', 'deblend_nPeaks', 'deblend_parentNPeaks', 'detect_isPrimary', 'detect_isDeblendedSource', 'detect_fromBlend', 'detect_isIsolated', 'base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_SdssCentroid_xErr', 'base_SdssCentroid_yErr', 'base_SdssCentroid_flag', 'base_CircularApertureFlux_flag_badCentroid', 'base_PsfFlux_flag_badCentroid', 'ext_shapeHSM_HsmPsfMoments_flag_badCentroid', 'ext_shapeHSM_HsmSourceMoments_flag_badCentroid', 'base_SdssCentroid_flag_edge', 'base_CircularApertureFlux_flag_badCentroid_edge', 'base_PsfFlux_